In [1]:
import pandas as pd
import unicodedata
import re
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

In [2]:
spark = SparkSession.builder.appName("ExcelProcessing").getOrCreate()

In [3]:
df = pd.read_excel(r"C:\Khue\TDN\data\raw\thong tin NMTD.xls")

In [4]:
# remove last row
df = df[:-1]
df

,DVDL,NMTD,MADIEMDO
0,CTY ĐIỆN LỰC BẮC KẠN,NMTĐ TÀ LÀNG,G2A121S000D371
1,CTY ĐIỆN LỰC BẮC KẠN,NMTĐ TÀ LÀNG,G2A121S000M371
2,CTY ĐIỆN LỰC BẮC KẠN,NMTĐ THƯỢNG ÂN,G2A122S000D371
3,CTY ĐIỆN LỰC BẮC KẠN,NMTĐ THƯỢNG ÂN,G2A122S000M371
4,CTY ĐIỆN LỰC BẮC KẠN,NMTĐ NẬM CẮT,G2A123S000M371
...,...,...,...
802,CTY ĐIỆN LỰC TUYÊN QUANG,NMTĐ Sông Lô 8A,G2A203S000D131
803,CTY ĐIỆN LỰC TUYÊN QUANG,NMTĐ Sông Lô 8B,G2A214S000D132
804,CTY ĐIỆN LỰC TUYÊN QUANG,NMTĐ Sông Lô 8B,G2A214S000M132
805,CTY ĐIỆN LỰC TUYÊN QUANG,NMTĐ Sông Lô 8B,G2A214S000D131


In [5]:
df = spark.createDataFrame(df)

In [6]:
df.show()
df.printSchema()

+--------------------+-----------------+--------------+
|                DVDL|             NMTD|      MADIEMDO|
+--------------------+-----------------+--------------+
|CTY ĐIỆN LỰC BẮC KẠN|     NMTĐ TÀ LÀNG|G2A121S000D371|
|CTY ĐIỆN LỰC BẮC KẠN|     NMTĐ TÀ LÀNG|G2A121S000M371|
|CTY ĐIỆN LỰC BẮC KẠN|   NMTĐ THƯỢNG ÂN|G2A122S000D371|
|CTY ĐIỆN LỰC BẮC KẠN|   NMTĐ THƯỢNG ÂN|G2A122S000M371|
|CTY ĐIỆN LỰC BẮC KẠN|     NMTĐ NẬM CẮT|G2A123S000M371|
|CTY ĐIỆN LỰC BẮC KẠN|     NMTĐ NẬM CẮT|G2A123S000D371|
|CTY ĐIỆN LỰC BẮC KẠN|     THÁC GIỀNG 1|G2A219S000D371|
|CTY ĐIỆN LỰC BẮC KẠN|     THÁC GIỀNG 1|G2A219S000M371|
|CTY ĐIỆN LỰC BẮC KẠN|     NMTĐ PÁC CÁP|G2A234S000D331|
|CTY ĐIỆN LỰC BẮC KẠN|     NMTĐ PÁC CÁP|G2A234S000M331|
|CTY ĐIỆN LỰC BẮC KẠN|  NMTĐ Khuổi Thốc|G2A300S000D371|
|CTY ĐIỆN LỰC BẮC KẠN|  NMTĐ Khuổi Thốc|G2A300S000M371|
|CTY ĐIỆN LỰC CAO ...|      NMTĐ BẢN RẠ|G2A103S000D174|
|CTY ĐIỆN LỰC CAO ...|      NMTĐ BẢN RẠ|G2A103S000M174|
|CTY ĐIỆN LỰC CAO ...|   NMTĐ BẢN HOÀNG|G2A099S0

In [7]:
def transform_name(name):
    if name.startswith("NMTĐ "):
        name = name[len("NMTĐ ") :]
    name = unicodedata.normalize("NFKD", name).encode("ascii", "ignore").decode("utf-8")
    name = re.sub(r"\s+", "_", name.strip())
    return name.upper()

In [8]:
# Đăng ký UDF
transform_name_udf = udf(transform_name, StringType())

In [ ]:
# Tạo 1 cột mới giống cột nhà máy và Áp dụng UDF lên cột tên nhà máy
df = df.withColumn("NMTD", transform_name_udf(df["NMTD"]))

In [10]:
df.show()

+--------------------+------------+--------------+
|                DVDL|        NMTD|      MADIEMDO|
+--------------------+------------+--------------+
|CTY ĐIỆN LỰC BẮC KẠN|     TA_LANG|G2A121S000D371|
|CTY ĐIỆN LỰC BẮC KẠN|     TA_LANG|G2A121S000M371|
|CTY ĐIỆN LỰC BẮC KẠN|   THUONG_AN|G2A122S000D371|
|CTY ĐIỆN LỰC BẮC KẠN|   THUONG_AN|G2A122S000M371|
|CTY ĐIỆN LỰC BẮC KẠN|     NAM_CAT|G2A123S000M371|
|CTY ĐIỆN LỰC BẮC KẠN|     NAM_CAT|G2A123S000D371|
|CTY ĐIỆN LỰC BẮC KẠN|THAC_GIENG_1|G2A219S000D371|
|CTY ĐIỆN LỰC BẮC KẠN|THAC_GIENG_1|G2A219S000M371|
|CTY ĐIỆN LỰC BẮC KẠN|     PAC_CAP|G2A234S000D331|
|CTY ĐIỆN LỰC BẮC KẠN|     PAC_CAP|G2A234S000M331|
|CTY ĐIỆN LỰC BẮC KẠN|  KHUOI_THOC|G2A300S000D371|
|CTY ĐIỆN LỰC BẮC KẠN|  KHUOI_THOC|G2A300S000M371|
|CTY ĐIỆN LỰC CAO ...|      BAN_RA|G2A103S000D174|
|CTY ĐIỆN LỰC CAO ...|      BAN_RA|G2A103S000M174|
|CTY ĐIỆN LỰC CAO ...|   BAN_HOANG|G2A099S000D371|
|CTY ĐIỆN LỰC CAO ...|   BAN_HOANG|G2A099S000M371|
|CTY ĐIỆN LỰC CAO ...|      NA_

In [11]:
df = df.toPandas()
# Lưu dữ liệu xuống file excel
df.to_excel(r"C:\Khue\TDN\data\raw\thong_tin_NMTD_convert.xlsx", index=False)

In [39]:
# Xoá những mã điểm đo có dạng khác dạng: "G2A234S000M331"
df = df[df["MADIEMDO"].str.match(r"G\d+A\d+S\d+M\d+")]

df

# Lưu dữ liệu xuống file excel
df.to_excel(r"C:\Khue\TDN\data\raw\thong_tin_NMTD_main.xlsx", index=False)

In [33]:
file_path = r"C:\Khue\TDN\data\raw\sanluongipp-2021.xls"
excel_file = pd.ExcelFile(file_path)
sheet_names = excel_file.sheet_names
print("Số lượng sheet trong file:", len(sheet_names))

Số lượng sheet trong file: 67


In [34]:
# Các dữ liệu độc nhất trong cột DVDL
unique_dvdl = df.select("DVDL").distinct().collect()
unique_dvdl = [row["DVDL"] for row in unique_dvdl]
print("Các dữ liệu độc nhất trong cột DVDL:", unique_dvdl)

Các dữ liệu độc nhất trong cột DVDL: ['CTY ĐIỆN LỰC HÀ GIANG', 'CTY ĐIỆN LỰC ĐIỆN BIÊN', 'CTY ĐIỆN LỰC CAO BẰNG', 'CTY ĐIỆN LỰC BẮC KẠN', 'CTY ĐIỆN LỰC HÒA BÌNH', 'CTY ĐIỆN LỰC HÀ TĨNH', 'CTY ĐIỆN LỰC LAI CHÂU', 'CTY ĐIỆN LỰC LÀO CAI', 'CTY ĐIỆN LỰC LẠNG SƠN', 'CTY ĐIỆN LỰC SƠN LA', 'CTY ĐIỆN LỰC NGHỆ AN', 'CTY ĐIỆN LỰC QUẢNG NINH', 'CTY ĐIỆN LỰC TUYÊN QUANG', 'CTY ĐIỆN LỰC YÊN BÁI', 'CTY ĐIỆN LỰC THANH HÓA', 'CTY ĐIỆN LỰC THÁI NGUYÊN']


In [ ]:
import string


def remove_accents(input_str):
    # Chuẩn hóa Unicode về dạng NFKD
    nfkd_form = unicodedata.normalize("NFKD", input_str)
    # Loại bỏ các ký tự dấu (combining characters)
    return "".join([c for c in nfkd_form if not unicodedata.combining(c)])


def convert_company(name):
    # Loại bỏ tiền tố "CTY ĐIỆN LỰC " (nếu có)
    core_name = name.replace("CTY ĐIỆN LỰC ", "")
    # Loại bỏ dấu và chuyển sang chữ không dấu
    core_name = remove_accents(core_name)
    # Loại bỏ khoảng trắng
    core_name = core_name.replace(" ", "")
    # Thêm tiền tố "PC_"
    return "PC_" + core_name


# Áp dụng cho từng phần tử trong list
unique_dvdl = [convert_company(name) for name in unique_dvdl]

print(unique_dvdl)

['PC_HAGIANG', 'PC_ĐIENBIEN', 'PC_CAOBANG', 'PC_BACKAN', 'PC_HOABINH', 'PC_HATINH', 'PC_LAICHAU', 'PC_LAOCAI', 'PC_LANGSON', 'PC_SONLA', 'PC_NGHEAN', 'PC_QUANGNINH', 'PC_TUYENQUANG', 'PC_YENBAI', 'PC_THANHHOA', 'PC_THAINGUYEN']


In [15]:
df_TDN = pd.read_excel(r"C:\Khue\TDN\data\raw\DS_TTT_TDN.xlsx")
df_TDN

,ID_NM,DIEU_DO,TCTDL,DIEN_LUC,TENNHAMAY,TEN_NM,LOAI_HINH,QUYEN_DIEUKHIEN,CS_DAT,A_TB_NAM,U_DAUNOI,MIEN,TINH,HUYEN,KINHDO,VIDO,BACTHANGSONG,TD_THAMCHIEU,HIEU_LUC,DVPD
0,10997,A1,NPC,PC_BACKAN,Nặm Cắt,NAM_CAT,1,PC_BACKAN,3.2,0.000,35,Bắc,Bắc Kạn,Bạch Thông,105.789306,22.192518,SONG_HONG,TUYEN_QUANG,1,NaN
1,10998,A1,NPC,PC_BACKAN,Pác Cáp,PAC_CAP,1,PC_BACKAN,6.0,0.000,0,NaN,NaN,NaN,106.127303,22.210818,SONG_HONG,TUYEN_QUANG,1,NaN
2,10995,A1,NPC,PC_BACKAN,Tà Làng,TA_LANG,1,PC_BACKAN,4.5,0.000,0,NaN,NaN,NaN,105.721185,22.303931,SONG_HONG,TUYEN_QUANG,1,NaN
3,11193,A1,NPC,PC_BACKAN,THÁC GIỀNG 1,THAC_GIENG_1,1,PC_BACKAN,5.5,0.000,0,NaN,NaN,NaN,105.893735,22.095786,SONG_HONG,TUYEN_QUANG,1,NaN
4,11191,A1,NPC,PC_BACKAN,Thượng Ân,THUONG_AN,1,PC_BACKAN,2.4,0.000,0,NaN,NaN,NaN,106.003945,22.511839,SONG_HONG,TUYEN_QUANG,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532,11054,A1,NPC,PC_YENBAI,Thào Sa Chải,THAO_SA_CHAI,1,PC_YENBAI,6.5,22.684,35,Bắc,Yên Bái,Mù Cang Chải,104.268669,21.838929,SONG_HONG,BAN_CHAT,0,NaN
533,10830,A1,NPC,PC_YENBAI,Trạm Tấu,TRAM_TAU,1,A1,30.0,0.000,0,NaN,NaN,NaN,104.442036,21.498660,SONG_HONG,BAN_CHAT,1,NaN
534,10840,A1,NPC,PC_YENBAI,Vực Tuần,VUC_TUAN,1,PC_YENBAI,5.0,0.000,0,NaN,NaN,NaN,104.569095,21.416027,SONG_HONG,BAN_CHAT,1,NaN
535,10837,A1,NPC,PC_YENBAI,Nậm Tục,NAM_TUC,1,PC_YENBAI,3.0,0.000,0,NaN,NaN,NaN,104.447998,21.608201,SONG_HONG,BAN_CHAT,1,NaN


In [ ]:
# Lọc dữ liệu theo list cho trước
df_TDN = df_TDN[df_TDN["Mã ĐVQL"].isin(unique_dvdl)]
df_TDN